In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.stem import WordNetLemmatizer
import sqlite3
import string

In [5]:
conn = sqlite3.connect("../db_and_csvs/unt_clubs.db")
data = pd.read_sql("SELECT * FROM organizations ORDER BY name ASC LIMIT 1", conn)
data #test

,name,short_name,summary,description,picture,url
0,3D Printing Club,None,"Learn about, design, use, and maintain additiv...",This is the 3D printing Club! We are a group o...,https://se-images.campuslabs.com/clink/images/...,https://unt.campuslabs.com/engage/organization...


In [6]:
type(data)

pandas.core.frame.DataFrame

In [35]:
data = pd.read_sql_query("SELECT * FROM organizations", conn)
try:
    conn.close()
    print('db closed')
except Exception as e:
    print(f'db err: {e}')

db closed


In [38]:
data[:1]

,name,short_name,summary,description,picture,url
0,3D Printing Club,None,"Learn about, design, use, and maintain additiv...",This is the 3D printing Club! We are a group o...,https://se-images.campuslabs.com/clink/images/...,https://unt.campuslabs.com/engage/organization...


In [ ]:
# define lemmatizer
lemmatize = WordNetLemmatizer()

def lemmatize_text(text):
    text = text.lower().translate(str.maketrans('','', 'string.punctuation'))
    terms = text.split()
    return " ".join([lemmatize.lemmatize(word) for word in terms])

In [20]:
# define 'document' as described on the paper
# 2* weight on short_name
data.fillna('')
data['document'] = (
    data['name'] + " " + 
    data['short_name'].astype(str) + " " + data['short_name'].astype(str) + " " +
    data['summary'] + " " + 
    data['description']
)

In [62]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import WordNetLemmatizer
import nltk
import sqlite3
import string

def access_and_load_db():
    conn = sqlite3.connect("db_and_csvs/unt_clubs.db")
    data = pd.read_sql_query("SELECT * FROM organizations", conn)
    try:
        conn.close()
        print('db closed')
    except Exception as e:
        print(f'db err: {e}')
    # define 'document' as described on the paper
    # 2* weight on short_name
    data = data.fillna('')
    data['document'] = (
        data['name'] + " " + 
        data['short_name'].astype(str) + " " + data['short_name'].astype(str) + " " +
        data['summary'] + " " + 
        data['description']
    )
    return data


# define lemmatizer
lemmatize = WordNetLemmatizer()
def lemmatize_text(text):
    text = text.lower().translate(str.maketrans('','', string.punctuation))
    terms = text.split()
    return " ".join([lemmatize.lemmatize(word) for word in terms])


def get_recommendations(query, data, tfidf_vectorizer, tfidf_matrix):
    query_vectorized = tfidf_vectorizer.transform([query])
    cosine_sim = cosine_similarity(query_vectorized, tfidf_matrix)
    top_five = np.argsort(cosine_sim[0])[-5:][::-1]
    return data.iloc[top_five]

if __name__ == "__main__":
    data = access_and_load_db()

    stop_words = list(text.ENGLISH_STOP_WORDS)
    lemmatized_stop_words = [lemmatize.lemmatize(word) for word in stop_words]

    #https://www.geeksforgeeks.org/machine-learning/music-recommendation-system-using-machine-learning/
    #https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
    #ref
    tfidf = TfidfVectorizer(
        preprocessor=lemmatize_text,
        stop_words='english',
        token_pattern=r"\b\w\w+\b"
    )
    tfidf_matrix = tfidf.fit_transform(data['document'])

    usr_input = "i am asian and looking for new friends"
    res = get_recommendations(usr_input, data, tfidf, tfidf_matrix)

    print(f'top 5 matches:')
    res_display = res[['name', 'short_name']].reset_index(drop=True)
    res_display.index = res_display.index + 1
    print(res_display)
    

db closed
top 5 matches:
                                        name short_name
1                  Asian Student Association        ASA
2  Society of Asian Scientists and Engineers   UNT SASE
3              SALT South Asian InterVarsity  SALT SAIV
4                North Texas Pickleball Club       NTPC
5         Delta Epsilon Psi Fraternity, Inc.      DEPsi


C:\Users\jtras\AppData\Roaming\Python\Python313\site-packages\sklearn\feature_extraction\text.py:411: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'wa'] not in stop_words.
  warnings.warn(


In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import WordNetLemmatizer
import nltk
import sqlite3
import string

SYNONYM_MAP = {
    "acct": "accounting",
    "aero": "aerospace engineering",
    "african": "african",
    "ai": "artificial intelligence",
    "alliance": "alliance",
    "arab": "arab",
    "arch": "architecture",
    "art": "art",
    "asian": "asian",
    "assoc": "association",
    "athletic": "athletics",
    "award": "award",
    "badminton": "badminton",
    "basketball": "basketball",
    "bio": "biology",
    "bioeng": "biomedical engineering",
    "biz": "business",
    "brazilian": "brazilian",
    "campus": "campus ministry",
    "charity": "charity",
    "chem": "chemistry",
    "chinese": "chinese",
    "christian": "christian",
    "civil": "civil engineering",
    "club": "club",
    "club sports": "club sports",
    "cs": "computer science",
    "cse": "computer science engineering",
    "cultural": "cultural",
    "dance": "dance",
    "db": "database",
    "dent": "dental",
    "design": "design",
    "devops": "devops engineering",
    "diversity": "diversity",
    "econ": "economics",
    "elec": "electrical engineering",
    "eng": "english",
    "entrepreneurship": "entrepreneurship",
    "faith": "faith",
    "fellowship": "fellowship",
    "fin": "finance",
    "football": "football",
    "frat": "fraternity",
    "fullstack": "full stack development",
    "golf": "golf",
    "gov": "government",
    "graduate": "graduate",
    "greek": "greek life",
    "group": "group",
    "hist": "history",
    "honors": "honors",
    "hr": "human resources",
    "inclusion": "inclusion",
    "indian": "indian",
    "innovation": "innovation",
    "international": "international",
    "intl": "international",
    "intramural": "intramural",
    "it": "information technology",
    "japanese": "japanese",
    "korean": "korean",
    "lacrosse": "lacrosse",
    "latin": "latin american",
    "law": "law",
    "lgbtq": "lgbtq",
    "lgbtqia": "lgbtq",
    "lit": "literature",
    "mech": "mechanical engineering",
    "med": "medicine",
    "mexican": "mexican",
    "mgmt": "management",
    "ministry": "ministry",
    "mkt": "marketing",
    "ml": "machine learning",
    "music": "music",
    "muslim": "muslim",
    "neuro": "neuroscience",
    "nsls": "national society of leadership and success",
    "nursing": "nursing",
    "org": "organization",
    "outreach": "community outreach",
    "pharm": "pharmacy",
    "philanthropy": "philanthropy",
    "phys": "physics",
    "pickleball": "pickleball",
    "pol": "political science",
    "pre-med": "pre-medicine",
    "pre-vet": "pre-veterinary medicine",
    "professional": "professional",
    "psych": "psychology",
    "pt": "physical therapy",
    "pta": "physical therapy assistant",
    "rotc": "reserve officers training corps",
    "rugby": "rugby",
    "sales": "sales",
    "sci": "science",
    "service": "community service",
    "soc": "society",
    "soccer": "soccer",
    "social": "social",
    "sorority": "sorority",
    "sports": "sports",
    "sql": "database",
    "student": "student",
    "swe": "software engineering",
    "swimming": "swimming",
    "tennis": "tennis",
    "theater": "theatre",
    "ttu": "texas tech university",
    "unt": "university of north texas",
    "ut": "university of texas",
    "vet": "veterinary",
    "volleyball": "volleyball",
    "volunteer": "volunteering",
    "web": "web development",
}

def access_and_load_db():
    conn = sqlite3.connect("db_and_csvs/unt_clubs.db")
    data = pd.read_sql_query("SELECT * FROM organizations", conn)
    try:
        conn.close()
        print('db closed')
    except Exception as e:
        print(f'db err: {e}')
    # define 'document' as described on the paper
    # increase weight on short_name
    data = data.fillna('')
    data['document'] = (
        data['name'] + " " + 
        data['short_name'].astype(str) + " " + data['short_name'].astype(str) + " " +
        data['summary'] + " " + 
        data['description']
    )
    #print(type(data))
    return data


# define lemmatizer
lemmatize = WordNetLemmatizer()
def lemmatize_text(text):
    text = text.lower().translate(str.maketrans('','', string.punctuation))
    terms = text.split()
    new_words = [SYNONYM_MAP.get(word, word) for word in terms]

    return " ".join([lemmatize.lemmatize(word) for word in new_words])


def get_recommendations(query, data, tfidf_vectorizer, tfidf_matrix):
    query_vectorized = tfidf_vectorizer.transform([query])
    cosine_sim = cosine_similarity(query_vectorized, tfidf_matrix)
    top_five = np.argsort(cosine_sim[0])[-5:][::-1]
    return data.iloc[top_five]

if __name__ == "__main__":
    try:
        data = access_and_load_db()

        stop_words = list(text.ENGLISH_STOP_WORDS)
        lemmatized_stop_words = [lemmatize.lemmatize(word) for word in stop_words]

        tfidf = TfidfVectorizer(
            preprocessor=lemmatize_text,
            stop_words=lemmatized_stop_words,
            token_pattern=r"\b\w\w+\b"
        )
        tfidf_matrix = tfidf.fit_transform(data['document'])

        usr_input = input("Enter what you're looking for in an organization: ")
        res = get_recommendations(usr_input, data, tfidf, tfidf_matrix)

        print(f'top 5 matches for {usr_input}:')
        res_display = res[['name', 'short_name']].reset_index(drop=True)
        res_display.index = res_display.index + 1
        print(res_display)
    except Exception as e:
        print(f'error: {e}')

error: unable to open database file
